In [1]:
import sys
sys.path.insert(0, '..')

import torch
import os
import wandb
import random
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from ignite.handlers.param_scheduler import create_lr_scheduler_with_warmup
from torch.utils.data import DataLoader
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

from core.final.dataset import PSMDataset
from core.final.model import GalSpecNet, MetaModel, Informer, AstroModel
from core.final.trainer import Trainer

/global/home/users/mariia/miniforge3/envs/astro/lib/python3.10/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [9]:
plt.ioff()

In [10]:
def get_model(config):
    if config['mode'] == 'photo':
        model = Informer(config)
    elif config['mode'] == 'spectra':
        model = GalSpecNet(config)
    elif config['mode'] == 'meta':
        model = MetaModel(config)
    else:
        model = AstroModel(config)

    return model

In [11]:
def calc_results(run_id, last_epoch):
    api = wandb.Api()
    run = api.run(run_id)
    config = run.config
    
    config['use_wandb'] = False
    config['file'] = 'preprocessed_data/full_lb/spectra_and_v'
    
    test_dataset = PSMDataset(config, split='test')
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = get_model(config)
    model = model.to(device)
    
    weights_path = os.path.join(config['weights_path'] + '-' + run_id.split('/')[-1], f'weights-{last_epoch}.pth')
    model.load_state_dict(torch.load(weights_path, weights_only=False))

    trainer = Trainer(model=model, optimizer=None, scheduler=None, warmup_scheduler=None, criterion=None, device=device, config=config)
    conf_matrix = trainer.evaluate(test_dataloader, test_dataset.id2target)
    conf_matrix_percent = 100 * conf_matrix / conf_matrix.sum(axis=1)[:, np.newaxis]
    
    acc = np.trace(conf_matrix) / np.sum(conf_matrix)
    acc_percent = np.trace(conf_matrix_percent) / np.sum(conf_matrix_percent)
    
    return acc, acc_percent

In [12]:
sub50 = {
    'photo': ['uwi549ko', 'lasio95e', 'ssjgsf4o', '8pjmji36', 'ey93jptd'],
    'spectra': ['9w2iy7if', 'uxdgxzzg', '20jf5fc3', 'apy0hwze', 'l8r1v87z'],
    'meta': ['dnql8zxb', 'w4z8mc7g', '872t6r9u', 'pdk8lhmr', 'tiorlnpa'],
    'all': ['awnlvghm', 'u7lhipcv', 'ibj38qtl', 'v268pfls', '10k7n8f8'],
    'photoclip': ['6cfb67n2', 'u2xrn2es', '7ds9f4j0', 'yoel2uju', 'klhhmrgz'],
    'spectraclip': ['jft4o7ll', '2ys1pu74', '0u83wkoo', 'ubyfjbl6', '6mhe95eb'],
    'metaclip': ['6afyvxnq', 'f4lt4n3m', '1hlifayz', '30m8j9kv', 'a2pps0vg'],
    'allclip': ['52fk0181', 'f4hvk7s9', 'xm2gkm0g', 'sf3f1fn7', '80s6j269']
}

sub25 = {
    'photo': ['i16e091u', 'yq6wrwn8', 'khf2rzs1', '32po6rjv', '3oz9g756'],
    'spectra': ['85x94bpe', 'k4pywvjv', 'dy7uliaw', 's66ifk69', 'rn3x84v9'],
    'meta': ['5n21zp2m', 'e2jkak5c', '02zpe2gy', 'dado4tth', 'etn9tbcy'],
    'all': ['wit0n3w3', 'w3d0xbe2', '96fwt2gx', 'du5zsfz6', '0k73x5qd'],
    'photoclip': ['1xgyaljv', '35idhf4w', 'o8n5dti7', 'fhpmo60y', '4at8i8ji'],
    'spectraclip': ['x1ec0yff', 'd5c3rhwz', 'kggejkom', '52xgy4wt', 'r5ctmuri'],
    'metaclip': ['osxq73a0', 'p2ozjutp', 'sce7zk10', 'bw5lzst5', '8hhqrs6d'],
    'allclip': ['a5peiq96', '05o27w72', 'xnbiygd1', '71qz3y5q', 'j8bnyk18']
}

sub10 = {
    'photo': ['nq1bhy0w', 'ogd2l28v', 'yagx8pk7', 'eh9qmriw', 'mk9ckjb4'],
    'spectra': ['vvljqb9o', 'bjnix390', 'wwrv3nut', 'pagk0jo5', '2dx7j4sy'],
    'meta': ['qqjg8qk3', 'n38nru7q', 'xftkn3gw', 'dtcx3xqk', 'r5mqe2rp'],
    'all': ['8hl6cpj7', 'h01wkom5', '5ivkbdrq', '5l9h32ak', 'thf3hzty'],
    'photoclip': ['v9hs5mvj', 'vc0tj3ee', '451pr1oc', 'lit1jza0', 'xp3lxwkn'],
    'spectraclip': ['x27wg2o5', 'gbp5cbc4', '0t8y55t8', 'hu285ze4', '3xe7g7ac'],
    'metaclip': ['0sjxdzrr', 'fm8wkcdn', 'vsku583d', '68n5fsm7', 'plpa1d4x'],
    'allclip': ['h4bq7juh', '7l1h5ak1', '6apkn9rt', 'hqwxn9df', 'eeur7zbf']
}

In [13]:
meta_sub_acc = {
    'meta': {
        'sub50': [],
        'sub25': [],
        'sub10': []
    },
    'metaclip': {
        'sub50': [],
        'sub25': [],
        'sub10': []
    }
}

for mode in meta_sub_acc:
    for s in meta_sub_acc[mode]:
        print(mode, s)
        
        if s == 'sub50':
            runs = sub50[mode]
        elif s == 'sub25':
            runs = sub25[mode]
        elif s == 'sub10':
            runs = sub10[mode]

        for r in runs:
            acc, acc_percent = calc_results('MeriDK/AstroCLIPResults3/' + r, 'best')
            meta_sub_acc[mode][s].append((acc, acc_percent))

meta sub50


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.01it/s]


meta sub25


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.02it/s]


meta sub10


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.01it/s]


metaclip sub25


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.02it/s]
/global/home/users/mariia/AstroML/notebooks/../core/final/trainer.py:248: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 7))
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.92it/s]


metaclip sub10


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  5.00it/s]


In [14]:
photo_sub_acc = {
    'photo': {
        'sub50': [],
        'sub25': [],
        'sub10': []
    },
    'photoclip': {
        'sub50': [],
        'sub25': [],
        'sub10': []
    }
}

for mode in photo_sub_acc:
    for s in photo_sub_acc[mode]:
        print(mode, s)
        
        if s == 'sub50':
            runs = sub50[mode]
        elif s == 'sub25':
            runs = sub25[mode]
        elif s == 'sub10':
            runs = sub10[mode]

        for r in runs:
            acc, acc_percent = calc_results('MeriDK/AstroCLIPResults3/' + r, 'best')
            photo_sub_acc[mode][s].append((acc, acc_percent))

photo sub50


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.61s/it]


photo sub25


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.64s/it]


photo sub10


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.62s/it]


photoclip sub25


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.79s/it]


photoclip sub10


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.61s/it]


In [21]:
spectra_sub_acc = {
    'spectra': {
        'sub50': [],
        'sub25': [],
        'sub10': []
    },
    'spectraclip': {
        'sub50': [],
        'sub25': [],
        'sub10': []
    }
}

for mode in spectra_sub_acc:
    for s in spectra_sub_acc[mode]:
        print(mode, s)
        
        if s == 'sub50':
            runs = sub50[mode]
        elif s == 'sub25':
            runs = sub25[mode]
        elif s == 'sub10':
            runs = sub10[mode]

        for r in runs:
            acc, acc_percent = calc_results('MeriDK/AstroCLIPResults3/' + r, 'best')
            spectra_sub_acc[mode][s].append((acc, acc_percent))

spectra sub50


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:24<00:00,  4.81s/it]


spectra sub25


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:23<00:00,  4.61s/it]


spectra sub10


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.46s/it]


spectraclip sub25


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.47s/it]


spectraclip sub10


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.59s/it]


In [25]:
all_sub_acc = {
    'all': {
        'sub50': [],
        'sub25': [],
        'sub10': []
    },
    'allclip': {
        'sub50': [],
        'sub25': [],
        'sub10': []
    }
}

for mode in all_sub_acc:
    for s in all_sub_acc[mode]:
        print(mode, s)
        
        if s == 'sub50':
            runs = sub50[mode]
        elif s == 'sub25':
            runs = sub25[mode]
        elif s == 'sub10':
            runs = sub10[mode]

        for r in runs:
            acc, acc_percent = calc_results('MeriDK/AstroCLIPResults3/' + r, 'best')
            all_sub_acc[mode][s].append((acc, acc_percent))

all sub50


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:31<00:00,  6.31s/it]


all sub25


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:30<00:00,  6.11s/it]


all sub10


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:30<00:00,  6.11s/it]


allclip sub25


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:30<00:00,  6.06s/it]


allclip sub10


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:30<00:00,  6.14s/it]


In [16]:
for mode in meta_sub_acc:
    for s in meta_sub_acc[mode]:   
        acc = meta_sub_acc[mode][s][0]
        mean = np.mean(acc)
        std = np.std(acc)
        print(mode, s, 'mean', round(mean * 100, 3), 'std', round(std * 100, 3), 'min', round((mean - std) * 100, 3), 'max', round((mean + std) * 100, 3))

meta sub50 mean 82.591 std 0.645 min 81.945 max 83.236
meta sub25 mean 79.975 std 0.604 min 79.371 max 80.58
meta sub10 mean 76.188 std 0.548 min 75.64 max 76.736
metaclip sub50 mean 83.511 std 0.399 min 83.112 max 83.91
metaclip sub25 mean 83.218 std 0.692 min 82.526 max 83.91
metaclip sub10 mean 78.16 std 0.272 min 77.888 max 78.432


In [17]:
for mode in meta_sub_acc:
    for s in meta_sub_acc[mode]:   
        acc = meta_sub_acc[mode][s][1]
        mean = np.mean(acc)
        std = np.std(acc)
        print(mode, 'weighted', s, 'mean', round(mean * 100, 3), 'std', round(std * 100, 3), 'min', round((mean - std) * 100, 3), 'max', round((mean + std) * 100, 3))

meta weighted sub50 mean 82.5 std 1.455 min 81.046 max 83.955
meta weighted sub25 mean 79.673 std 0.077 min 79.596 max 79.75
meta weighted sub10 mean 75.636 std 1.344 min 74.292 max 76.979
metaclip weighted sub50 mean 83.568 std 0.027 min 83.541 max 83.596
metaclip weighted sub25 mean 83.087 std 0.075 min 83.011 max 83.162
metaclip weighted sub10 mean 78.037 std 2.172 min 75.865 max 80.21


In [18]:
for mode in photo_sub_acc:
    for s in photo_sub_acc[mode]:
        acc = photo_sub_acc[mode][s][0]
        mean = np.mean(acc)
        std = np.std(acc)
        print(mode, s, 'mean', round(mean * 100, 3), 'std', round(std * 100, 3), 'min', round((mean - std) * 100, 3), 'max', round((mean + std) * 100, 3))

photo sub50 mean 87.217 std 0.34 min 86.876 max 87.557
photo sub25 mean 71.033 std 0.471 min 70.562 max 71.504
photo sub10 mean 68.772 std 3.543 min 65.23 max 72.315
photoclip sub50 mean 90.236 std 1.068 min 89.169 max 91.304
photoclip sub25 mean 88.331 std 1.095 min 87.236 max 89.426
photoclip sub10 mean 88.218 std 1.701 min 86.517 max 89.918


In [20]:
for mode in photo_sub_acc:
    for s in photo_sub_acc[mode]:
        acc = photo_sub_acc[mode][s][1]
        mean = np.mean(acc)
        std = np.std(acc)
        print(mode, 'weighted', s, 'mean', round(mean * 100, 3), 'std', round(std * 100, 3), 'min', round((mean - std) * 100, 3), 'max', round((mean + std) * 100, 3))

photo weighted sub50 mean 86.156 std 1.302 min 84.854 max 87.458
photo weighted sub25 mean 59.581 std 6.772 min 52.809 max 66.352
photo weighted sub10 mean 78.574 std 4.057 min 74.517 max 82.632
photoclip weighted sub50 mean 88.811 std 1.305 min 87.506 max 90.116
photoclip weighted sub25 mean 89.971 std 0.802 min 89.169 max 90.773
photoclip weighted sub10 mean 87.912 std 2.429 min 85.483 max 90.342


In [23]:
for mode in spectra_sub_acc:
    for s in spectra_sub_acc[mode]:
        acc = spectra_sub_acc[mode][s][0]
        mean = np.mean(acc)
        std = np.std(acc)
        print(mode, s, 'mean', round(mean * 100, 3), 'std', round(std * 100, 3), 'min', round((mean - std) * 100, 3), 'max', round((mean + std) * 100, 3))

spectra sub50 mean 68.046 std 5.572 min 62.474 max 73.618
spectra sub25 mean 68.34 std 1.009 min 67.331 max 69.348
spectra sub10 mean 45.939 std 0.757 min 45.182 max 46.697
spectraclip sub50 mean 70.556 std 4.456 min 66.1 max 75.011
spectraclip sub25 mean 68.34 std 3.301 min 65.039 max 71.64
spectraclip sub10 mean 63.091 std 0.909 min 62.182 max 64.0


In [24]:
for mode in spectra_sub_acc:
    for s in spectra_sub_acc[mode]:
        acc = spectra_sub_acc[mode][s][1]
        mean = np.mean(acc)
        std = np.std(acc)
        print(mode, 'weighted', s, 'mean', round(mean * 100, 3), 'std', round(std * 100, 3), 'min', round((mean - std) * 100, 3), 'max', round((mean + std) * 100, 3))

spectra weighted sub50 mean 70.593 std 4.103 min 66.49 max 74.697
spectra weighted sub25 mean 68.208 std 2.264 min 65.944 max 70.472
spectra weighted sub10 mean 45.948 std 0.105 min 45.843 max 46.053
spectraclip weighted sub50 mean 73.385 std 3.829 min 69.556 max 77.213
spectraclip weighted sub25 mean 72.901 std 2.11 min 70.791 max 75.011
spectraclip weighted sub10 mean 61.994 std 0.703 min 61.291 max 62.697


In [26]:
for mode in all_sub_acc:
    for s in all_sub_acc[mode]:
        acc = all_sub_acc[mode][s][0]
        mean = np.mean(acc)
        std = np.std(acc)
        print(mode, s, 'mean', round(mean * 100, 3), 'std', round(std * 100, 3), 'min', round((mean - std) * 100, 3), 'max', round((mean + std) * 100, 3))

all sub50 mean 92.152 std 0.063 min 92.09 max 92.215
all sub25 mean 91.289 std 0.727 min 90.562 max 92.017
all sub10 mean 87.657 std 1.32 min 86.337 max 88.977
allclip sub50 mean 90.949 std 0.163 min 90.787 max 91.112
allclip sub25 mean 91.374 std 0.048 min 91.326 max 91.422
allclip sub10 mean 88.991 std 0.946 min 88.045 max 89.938


In [27]:
for mode in all_sub_acc:
    for s in all_sub_acc[mode]:
        acc = all_sub_acc[mode][s][1]
        mean = np.mean(acc)
        std = np.std(acc)
        print(mode, 'weighted', s, 'mean', round(mean * 100, 3), 'std', round(std * 100, 3), 'min', round((mean - std) * 100, 3), 'max', round((mean + std) * 100, 3))

all weighted sub50 mean 94.644 std 0.442 min 94.202 max 95.086
all weighted sub25 mean 91.846 std 1.06 min 90.787 max 92.906
all weighted sub10 mean 87.537 std 0.66 min 86.876 max 88.197
allclip weighted sub50 mean 94.44 std 1.137 min 93.303 max 95.577
allclip weighted sub25 mean 92.28 std 1.674 min 90.607 max 93.954
allclip weighted sub10 mean 89.983 std 0.545 min 89.438 max 90.528
